# Resolviendo el modelo de Ising y el problema del corte máximo con un ordenador de D-Wave

En este notebook, vamos a utilizar un ordenador cuántico de D-Wave para resolver casos del modelo de Ising que se corresponden con instancias del problema del corte máximo en grafos. 

El hamiltoniano del modelo de Ising es $$H = \sum_{i,j=1}^n J_{i,j}Z_iZ_j + \sum_{i=1}^n h_iZ_i$$ 

En el caso particular del problema del corte máximo, se tiene $J_{i,j}=1$ y $h_i=0$ para todos los valores $i,j$.



Definir el modelo con el que vamos a trabajar es muy sencillo: se reduce a especificar los valores de las conexiones entre pares de qubits y los coeficientes $h_i$. Por ejemplo:

In [1]:
import numpy as np
import dimod

J = {(0,1):1}

h = {}
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

print(model)


BinaryQuadraticModel({0: 0, 1: 0}, {(0, 1): 1}, 0.0, Vartype.SPIN)


Podemos resolver el modelo de forma exacta

In [2]:
from dimod.reference.samplers import ExactSolver
sampler = ExactSolver()
solution = sampler.sample(model)
print(solution)

   0  1 energy num_oc.
1 +1 -1   -1.0       1
3 -1 +1   -1.0       1
0 -1 -1    1.0       1
2 +1 +1    1.0       1
['SPIN', 4 rows, 4 samples, 2 variables]


O con *simulated annealing* (un método heurístico de optimización para ordenadores clásicos)

In [4]:
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print(response)

   0  1 energy num_oc.
0 +1 -1   -1.0       1
1 -1 +1   -1.0       1
2 +1 -1   -1.0       1
3 -1 +1   -1.0       1
4 +1 -1   -1.0       1
5 -1 +1   -1.0       1
6 +1 -1   -1.0       1
7 +1 -1   -1.0       1
8 -1 +1   -1.0       1
9 -1 +1   -1.0       1
['SPIN', 10 rows, 10 samples, 2 variables]


Y, por supuesto, con el ordenador cuántico de D-Wave (requiere registro online en https://cloud.dwavesys.com/leap/)

In [7]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(model, num_reads=5000)
print(response)

   0  1 energy num_oc. chain_.
0 -1 +1   -1.0    1912     0.0
1 +1 -1   -1.0    3084     0.0
2 -1 -1    1.0       1     0.0
3 +1 +1    1.0       3     0.0
['SPIN', 4 rows, 5000 samples, 2 variables]


Veamos ahora un caso un poco más complicado, que se corresponde con encontrar un corte máximo en el grafo de la figura

<img src="max-cut.png" width="60%">

In [9]:
J = {(0,1):1,(0,2):1,(1,2):1,(1,3):1,(2,4):1,(3,4):1}
h = {}
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)

Primero lo resolvemos de forma exacta

In [10]:
sampler = ExactSolver()
solution = sampler.sample(model)
print(solution)

    0  1  2  3  4 energy num_oc.
8  -1 -1 +1 +1 -1   -4.0       1
9  +1 -1 +1 +1 -1   -4.0       1
28 -1 +1 -1 -1 +1   -4.0       1
29 +1 +1 -1 -1 +1   -4.0       1
4  -1 +1 +1 -1 -1   -2.0       1
11 -1 +1 +1 +1 -1   -2.0       1
14 +1 -1 -1 +1 -1   -2.0       1
17 +1 -1 -1 +1 +1   -2.0       1
27 -1 +1 +1 -1 +1   -2.0       1
30 +1 -1 -1 -1 +1   -2.0       1
2  +1 +1 -1 -1 -1    0.0       1
3  -1 +1 -1 -1 -1    0.0       1
6  +1 -1 +1 -1 -1    0.0       1
7  -1 -1 +1 -1 -1    0.0       1
12 -1 +1 -1 +1 -1    0.0       1
13 +1 +1 -1 +1 -1    0.0       1
18 +1 +1 -1 +1 +1    0.0       1
19 -1 +1 -1 +1 +1    0.0       1
22 +1 -1 +1 +1 +1    0.0       1
23 -1 -1 +1 +1 +1    0.0       1
24 -1 -1 +1 -1 +1    0.0       1
25 +1 -1 +1 -1 +1    0.0       1
1  +1 -1 -1 -1 -1    2.0       1
5  +1 +1 +1 -1 -1    2.0       1
10 +1 +1 +1 +1 -1    2.0       1
15 -1 -1 -1 +1 -1    2.0       1
16 -1 -1 -1 +1 +1    2.0       1
20 -1 +1 +1 +1 +1    2.0       1
26 +1 +1 +1 -1 +1    2.0       1
31 -1 -1 -

Ahora, con *simulated annealing*

In [11]:
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print(response)

   0  1  2  3  4 energy num_oc.
0 +1 -1 +1 +1 -1   -4.0       1
1 +1 +1 -1 -1 +1   -4.0       1
2 -1 +1 -1 -1 +1   -4.0       1
3 +1 -1 +1 +1 -1   -4.0       1
4 -1 -1 +1 +1 -1   -4.0       1
5 +1 +1 -1 -1 +1   -4.0       1
6 -1 +1 -1 -1 +1   -4.0       1
7 +1 +1 -1 -1 +1   -4.0       1
8 -1 -1 +1 +1 -1   -4.0       1
9 +1 -1 +1 +1 -1   -4.0       1
['SPIN', 10 rows, 10 samples, 5 variables]


Finalmente, lo resolvemos nuevamente con el *quantum annealer*

In [12]:
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(model, num_reads=5000)
print(response)

   0  1  2  3  4 energy num_oc. chain_.
0 -1 +1 -1 -1 +1   -4.0    1895     0.0
1 +1 +1 -1 -1 +1   -4.0    1152     0.0
2 +1 -1 +1 +1 -1   -4.0     726     0.0
3 -1 -1 +1 +1 -1   -4.0    1223     0.0
4 -1 +1 +1 -1 +1   -2.0       1     0.0
5 -1 +1 +1 +1 -1   -2.0       1     0.0
6 -1 +1 +1 -1 +1   -2.0       1     0.2
7 -1 +1 +1 -1 -1   -2.0       1     0.0
['SPIN', 8 rows, 5000 samples, 5 variables]
